In [ ]:
!pip install -q -U OpenAI

In [ ]:
#exclusive to colab environment. pip line for every session to install OpenAI
from google.colab import drive
drive.mount('/content/drive/')
!ls /content/drive/MyDrive/Leidos/demographic_generator/

Mounted at /content/drive/
BadGuy.py      DemoGenJuly.py	      demographics_sampling.py	generation.py	__pycache__
DemoGenAug.py  demographic_generator  female_names.txt		male_names.txt	surnames.txt


In [ ]:
from openai import OpenAI
import sys
import pickle
import threading
import copy
import json
import os
import time
import string
import random
string.punctuation += "¿¡"
#sys.path.append('../demographic_generator/') #FOR LINUX TYPE OS
sys.path.append('/content/drive/MyDrive/Leidos/demographic_generator/') #FOR GOOGLE COLAB
from DemoGenAug import demographicsLoader #from '' will be the file name of demographic .py file
os.environ["TZ"]="US/Central" #time modules used for generate unique filenames
time.tzset()

{'Name': ('Kendall', 'Dushkin'), 'Age': 21, 'Gender': 'man', 'Nationality': 'El Salvadoran', 'Items': 'an expired passport', 'Att': 'do not like authorities'}
{'Name': ('Jefferson', 'Karmely'), 'Age': 35, 'Gender': 'man', 'Nationality': 'Paraguayan', 'Suspicion': 'no personal possessions of any kind'}


In [ ]:
!pip install jdc # MODULE FOR HAVNNG CLASS FUNCTIONS ACROSS DIFFERENT CELLS
!pip install ipdb # MODULE FOR DEBUGGING/ADDING BREAKPOINT IN GOOGLE COLAB

  Using cached jedi-0.19.1-py2.py3-none-any.whl.metadata (22 kB)
Using cached jedi-0.19.1-py2.py3-none-any.whl (1.6 MB)


In [ ]:
import jdc
import ipdb # activate with: %pdb on
class self_talk:
  def __init__(self, top_p, temp, presence, PATH, key, freq=0.0):
    self.top_p = top_p
    self.temp = temp
    self.freq = freq
    self.presence = presence
    self.PATH = PATH

    self.client = OpenAI(api_key=key)
    self.df = demographicsLoader()

In [ ]:
    %%add_to self_talk
    def find_ngrams(self, input_list, n):
      return zip(*[input_list[i:] for i in range(n)])

In [ ]:
    %%add_to self_talk
    def cancel_batch(self, batch_id):
        """Utility to cancel a batch by ID"""
        self.client.batches.cancel(batch_id)

In [ ]:
    %%add_to self_talk
    def compute_metrics(self, x):
        """
        - N-grams and unique words
        """
        sequences = [y["content"] for y in x if y["role"] == "assistant"]
        unique_words = set()
        unique_ngrams = set()

        for utterance in sequences:
            utter = utterance.lower()

            for words in utter.split():
                unique_words.add(words.strip(string.punctuation))

            for i in range(4):
                for n in self.find_ngrams(utter.strip(string.punctuation).split(), i+1):
                    unique_ngrams.add(n)

        return unique_words, unique_ngrams

In [ ]:
    %%add_to self_talk
    def create_db(self, db_name : string, UIDs : list):
        """Convert multiple jsonl files to pickle object for easier transfer & loading"""

        self.change_dir()
        PATH = "../DB/" + db_name
        formatted = []


        for UID in UIDs:
            agent_name = f"agent_request_{UID}.jsonl"

            with open(agent_name, 'r', encoding='utf8') as f:
                for line in f:
                    line = json.loads(line)
                    formatted.append(line)

        with open(PATH, 'wb') as f:
            pickle.dump(formatted, f)

        print(f"Saved to file {PATH}")

In [ ]:
    %%add_to self_talk
    def timeout(self):
        time.sleep(150)

In [ ]:
    %%add_to self_talk
    def load_conversation(self, data):
        """
        - Filter format to determine if conversations stay in character
        """
        prompt = ""

        for i in range(len(data)-1):
            if (i + 1) % 2:
                prompt += "Person A: " + data[i+1]["content"] + "\n"
            else:
                prompt += "Person B: " + data[i+1]["content"] + "\n"

        return prompt

In [ ]:
    %%add_to self_talk
    def status(self, batch_id):
        return self.client.batches.retrieve(batch_id).status

In [ ]:
    %%add_to self_talk
    def append_jsonl(self, name, dictionary_list):
        """load string; add msgs; write over original file"""
        self.change_dir()

        formatted = []

        with open(name, 'r', encoding='utf8') as f:
            for line, generated in zip(f, dictionary_list):
                line = json.loads(line)
                line["body"]["messages"].append(generated["response"]["body"]["choices"][0]["message"])
                formatted.append(line)

        with open(name, 'w', encoding='utf8') as f:
            for line in formatted:
                json_record = json.dumps(line, ensure_ascii=False)
                f.write(json_record + '\n')

In [ ]:
    %%add_to self_talk
    def change_dir(self):
        """Change to batched dir"""
        print(os.getcwd())
        if os.getcwd() == self.PATH + "batched/":
            return

        os.chdir(self.PATH + "batched/")
        print("\n|\n" + os.getcwd())

In [ ]:
    %%add_to self_talk
    def print_content(self, name):
        """
        - utility function to print out batch file's context window
        """
        self.change_dir()

        with open(name, 'r', encoding='utf8') as f:
            for line in f:
                line = json.loads(line)

                for x in line["body"]["messages"]:
                    print(x["role"], "\t->\t", x["content"], "\n")
                print("\n"*2)

In [ ]:
    # NOT NEEDED TO RUN FOR BATCH PROCESSES. SINGLE CONVERSATION GENERATION METHOD.
    %%add_to self_talk
    def start_generation(self, tr=None, turns=2, utterances=12):
        """
        - Generic single API Requests
        - Can be threaded
        """
        accepted = 0
        rejections = 0
        tokens_in = 0
        tokens_out = 0


        for i in range(turns):

            print(f"----------------START OF TURN {tr} \t {i}----------------")

            start,notice,cond = self.convostarter()
            client_demographics = self.df.sample_client()
            agent_demographics = self.df.sample_agent()

            agent_dialogue, client_dialogue = self.generate_prompts2(start, notice, cond, client_demographics, agent_demographics)
            # PROMPT IS HERE^
            filter_system = f"""Person A's character is: "{agent_dialogue[0]["content"]}"

            Person B's character is: "{client_dialogue[0]["content"]}"

            Given Person A and Person B characters, determine if this conversation stays within character. Choose either yes or no."""


            chat_filter = [
                {"role": "system", "content": filter_system},
            ]


            print("\n\nClient: \t", client_dialogue, "\n\nAgent: \t", agent_dialogue, "\n\n")


            # print(f"\nBeginning of turn {i}.\n")

            # BEGINNING OF CONVERSATION
            for _ in range(utterances):

                # API Completion
                agent_out = self.client.chat.completions.create(
                    model="gpt-4o-mini",
                    messages=agent_dialogue,
                    top_p=self.top_p,
                    temperature=self.temp,
                    presence_penalty=self.presence,
                    frequency_penalty=self.freq
                )
                print(f"\tAgent output: {agent_out.choices[0].message.content}\n")

                # Addition to history
                agent_dialogue.append({"role":agent_out.choices[0].message.role, "content":agent_out.choices[0].message.content})
                client_dialogue.append({"role":"user", "content":agent_out.choices[0].message.content})

                # API Completion
                client_out = self.client.chat.completions.create(
                    model="gpt-4o-mini",
                    messages=client_dialogue,
                    top_p=self.top_p,
                    temperature=self.temp,
                    presence_penalty=self.presence,
                    frequency_penalty=self.freq
                )
                print(f"\tClient output: {client_out.choices[0].message.content}\n")
                # ADDED THIS 7-3-24
                Test = client_out.choices[0].message.content
                if "Adiós" in Test or "Hasta" in Test: continue
                # ADDED THIS 7-3-24
                # Addition to history
                agent_dialogue.append({"role":"user", "content":client_out.choices[0].message.content})
                client_dialogue.append({"role":client_out.choices[0].message.role, "content":client_out.choices[0].message.content})

                # Total tokens
                tokens_in += client_out.usage.prompt_tokens + agent_out.usage.prompt_tokens
                tokens_out += client_out.usage.completion_tokens + agent_out.usage.completion_tokens

            # Metrics for ngrams and unique words
            uw, un = self.compute_metrics(agent_dialogue)

            # Did it stay in character?
            chat_filter.append({"role": "user", "content": self.load_conversation(agent_dialogue)})

            filter_completion = self.client.chat.completions.create(
                model="gpt-4o-mini",
                messages=chat_filter
                )

            # Did it reach end of conversation
            EOC = self.client.chat.completions.create(
                model="gpt-4o-mini",
                messages=[
                    {"role": "system", "content": "At what point does this conversation occur? Choose either the beginning, middle, or end."},
                    {"role": "user", "content": self.load_conversation(agent_dialogue[-5:])}
                ]
                )

            metrics = {
                "Unique Words": uw,
                "Unique N-grams": un,
                "Character": filter_completion.choices[0].message.content,
                "End of Conversation": EOC.choices[0].message.content
                      }

            # print(f"\t\nUnique words: {len(metrics['Unique Words'])}\nUnique ngrams: {len(metrics['Unique N-grams'])}\n",
            #      f"Model stayed in character? {metrics['Character']}\n",
            #       f"Model reached EOC? {metrics['End of Conversation']}\n"
            #      )

            # print("Filter match?\t", ("end" in metrics["End of Conversation"].lower()), ("yes" in metrics["Character"].lower()))

            # Remember the agent turn will have all
            # if ("end" in metrics["End of Conversation"].lower() and "yes" in metrics["Character"].lower()):
            #     print(agent_dialogue)
            #     accepted += 1
            #     with open(self.PATH + f"accepted/Accepted_Turn_{tr}{i}_{round(time.time() * 1000)}", "wb") as f:
            #         pickle.dump({"Client History": client_dialogue,
            #                      "Agent History": agent_dialogue,
            #                      "Metrics": metrics
            #                     },
            #                     f)
            # else:
            #     # Rejection save
            #     rejections += 1
            #     with open(self.PATH + f"rejected/Rejected_Turn_{tr}{i}_{round(time.time() * 1000)}", "wb") as f:
            #         pickle.dump({"Client History": client_dialogue,
            #                      "Agent History": agent_dialogue,
            #                      "Metrics": metrics
            #                     },
            #                     f)


            print(f"{tr} TURN {i}\nTotal Accepted:Rejections {accepted}:{rejections}\nTotal tokens in/out: {tokens_in}/{tokens_out}\nSaved. Moving on to turn {i+1}\n", "\n"*4)
        print(f"{tr}: Final tokens in/out: {tokens_in}/{tokens_out}\n", "\n"*4)


In [ ]:
    %%add_to self_talk
    def generate_prompts(self, start, notice, cond, client_demographics, agent_demographics):
        """
        - Generates prompts for both the agent and client.
        """
        agent_dialogue = [
            {"role": "system", "content": f"You are an American airport customs agent. You pulled the user aside because of a suspicion about {agent_demographics['Suspicion']}. You notice they {client_demographics['Att']}{notice}. Ask the user questions in relation to airport security. Your mission is to determine if the user is suspicious from a airport security standpoint. If the user does not answer your questions, {cond}. Only speak in Spanish."},
            {"role": "user", "content": start}
        ]

        client_dialogue = [
            {"role": "system", "content": f"You are a {client_demographics['Age']}-year-old {client_demographics['Nationality']} {client_demographics['Gender']} visiting USA who has {client_demographics['Items']}. You {client_demographics['Att']}. A customs official has taken you aside for further inspection. Only respond in Spanish."},
        ]

        return agent_dialogue, client_dialogue

In [ ]:
    %%add_to self_talk
    def generate_prompts2(self, start, notice, cond, client_demographics, agent_demographics):
        """
        - Generates prompts for both the agent and client.
        """
        agent_dialogue = [
            {"role": "system", "content": f"""You are a customs agent at an American airport. You pulled the user aside because of a suspicion about {agent_demographics['Suspicion']}.
            You noticed they {client_demographics['Att']}{notice}. Ask the user questions related to airport security.
            Your mission is to determine if the user is suspicious from an airport security standpoint. If the user does not answer your questions, {cond}.
            Only speak in Spanish.

Here are some additional guidelines:
1. If the user apologizes or expresses an unmet need, briefly acknowledge the situation and then shift the topic to a relevant question about airport security. Use templates like:
   - "Entiendo. Ahora, por favor, dígame más sobre [tema relacionado con la seguridad]."
   - "No hay problema. ¿Puede explicarme [otro aspecto de seguridad]?"
   - "Gracias por informarme. Ahora, necesito saber sobre [acción sugerida]."

2. Remember that your main goal is to explore and discuss topics related to airport security. If the conversation gets stuck, reintroduce it with a new question or aspect of the topic.

3. Vary your responses to keep the conversation dynamic. If you feel you are repeating yourself, introduce a new topic or security question.

4. Remember to only speak in Spanish.

5. End the conversation with a friendly goodbye.

Every five exchanges, remind yourself of the main goal of the conversation: to determine if the user is suspicious from an airport security standpoint. If the conversation gets stuck, reset by introducing a new aspect of the topic or asking a new question.
"""},
            {"role": "user", "content": start}
        ]

        client_dialogue = [
            {"role": "system", "content": f"You are a {client_demographics['Age']}-year-old {client_demographics['Nationality']} {client_demographics['Gender']} visiting USA who has {client_demographics['Items']}. You {client_demographics['Att']}. A customs official has taken you aside for further inspection. Only respond in Spanish."},
        ]

        return agent_dialogue, client_dialogue

In [ ]:
    %%add_to self_talk
    def generate_prompts3(self, start, notice, cond, client_demographics, agent_demographics): # SPANISH PROMPTING, BUT FUNCTIONS TO ADD VARIABILITY ARE IN ENGLISH!!
        """
        - Generates prompts for both the agent and client.
        """
        agent_dialogue = [
            {"role": "system", "content": f"""Eres un agente de aduanas en un aeropuerto de los Estados Unidos. Detuviste al usuario debido a una sospecha sobre {agent_demographics['Suspicion']}. Notaste que {client_demographics['Att']}{notice}.
            Haz preguntas al usuario relacionadas con la seguridad del aeropuerto. Tu misión es determinar si el usuario es sospechoso desde el punto de vista de la seguridad del aeropuerto.
            Si el usuario no responde a tus preguntas, {cond}. Solo habla en español.

Aquí tienes algunas directrices adicionales:
1. Si el usuario se disculpa o expresa una necesidad no satisfecha, reconoce brevemente la situación y luego cambia el tema a una pregunta relevante sobre la seguridad del aeropuerto. Usa plantillas como:
   - "Entiendo. Ahora, por favor, dígame más sobre [tema relacionado con la seguridad]."
   - "No hay problema. ¿Puede explicarme [otro aspecto de seguridad]?"
   - "Gracias por informarme. Ahora, necesito saber sobre [acción sugerida]."

2. Recuerda que tu objetivo principal es explorar y discutir temas relacionados con la seguridad del aeropuerto. Si la conversación se estanca, reintrodúcelo con una nueva pregunta o aspecto del tema.

3. Varía tus respuestas para mantener la conversación dinámica. Si sientes que te estás repitiendo, introduce un nuevo tema o pregunta de seguridad.

Cada cinco intercambios, recuérdate a ti mismo el objetivo principal de la conversación: determinar si el usuario es sospechoso desde el punto de vista de la seguridad del aeropuerto. Si la conversación se atasca, restablece introduciendo un nuevo aspecto del tema o haciendo una nueva pregunta.
"""},
            {"role": "user", "content": start}
        ]

        client_dialogue = [
            {"role": "system", "content": f"You are a {client_demographics['Age']}-year-old {client_demographics['Nationality']} {client_demographics['Gender']} visiting USA who has {client_demographics['Items']}. You {client_demographics['Att']}. A customs official has taken you aside for further inspection. Only respond in Spanish."},
        ]

        return agent_dialogue, client_dialogue

In [ ]:
    %%add_to self_talk
    def create_batch(self, amt=1, start="Hello."): # VIRTUAL FILE IS BEING CREATED FOR EACH REQUEST, FOR BOTH AGENT AND CLIENT. - ML 07/03/24
        """
        - Creates starting batch based on starting phrase and amount of conversations you want to generate
        """
        self.change_dir()

        # Creation of jsonl
        agent_request = []
        client_request = []

        # File names
        UID = time.strftime('%m-%d-%y_%H%M-%S', time.localtime(time.time())) #CHANGED 8/4/24
        agent_name = f"agent_request_{UID}.jsonl"
        client_name = f"client_request_{UID}.jsonl"
        print(f"Requested {amt} prompts to be generated!")
        for i in range(amt):
            start,notice,cond = self.convostarter()
            client_demographics = self.df.sample_client()
            agent_demographics = self.df.sample_agent()

            agent_dialogue, client_dialogue = self.generate_prompts2(start, notice, cond, client_demographics, agent_demographics) # NOTE FUNCTION USED TO GENERATE PROMPTS!!
            print(f"Generated prompt: {i}!")
            client_out = self.client.chat.completions.create(model="gpt-4o-mini",messages=client_dialogue)

            agent_request.append({"custom_id": f"agent-request-{i}",
             "method": "POST",
             "url": "/v1/chat/completions",
             "body": {"model": "gpt-4o-mini",
                      "messages": agent_dialogue,
                      "top_p":self.top_p,
                      "temperature":self.temp,
                      "presence_penalty":self.presence,
                      "frequency_penalty":self.freq,
                      "max_tokens": 1000}})


            client_request.append({"custom_id": f"client-request-{i}",
             "method": "POST",
             "url": "/v1/chat/completions",
             "body": {"model": "gpt-4o-mini",
                      "messages": client_dialogue,
                      "top_p":self.top_p,
                      "temperature":self.temp,
                      "presence_penalty":self.presence,
                      "frequency_penalty":self.freq,
                      "max_tokens": 1000}})

        with open(agent_name, 'w', encoding='utf8') as f:
            for line in agent_request:
                json_record = json.dumps(line, ensure_ascii=False)
                f.write(json_record + '\n')

        with open(client_name, 'w', encoding='utf8') as f:
            for line in client_request:
                json_record = json.dumps(line, ensure_ascii=False)
                f.write(json_record + '\n')

        return UID

In [ ]:
    %%add_to self_talk
    def convostarter(self):
        """
        - Generates a random opening line for client to use. (ie. instead of always opening with Hola.)
        - For creating variability in prompts.
        """
        PhraseList = ["Hola.", "No respondo preguntas.", "¿Qué pasa?", "Hola, ¿qué tal?", "Buenos dias.", "Buenos tardes.", "Buenas noches.", "¿Cómo estás?", "¿Cómo te va?", "¿Cómo está usted?", "Adiós", "Hasta Luego."]
        phrase = random.choice(PhraseList)
        if random.randint(0,1):
            notice = ", but do not disclose this"
        else:
            notice = ''
        if random.randint(0,1):
            cond = "do not allow them to leave and call your supervisor"
        else:
            cond = "allow them to leave"
        return phrase, notice, cond

In [ ]:
    %pdb off # FOR TROUBLE SHOOTING

Incorrect argument. Use on/1, off/0, or nothing for a toggle.


In [ ]:
    %%add_to self_talk
    def generate_batches(self, UID, utterances=1):
        """
        - Submit the jsonl files
        - Retrieve the completion file from openai server
        - Extract generated output from retrieved file and put into format to append
        - Append generated output to jsonl file and continue generation
        """
        self.change_dir()
        agent_name = f"agent_request_{UID}.jsonl"
        client_name = f"client_request_{UID}.jsonl"

        for i in range(utterances):
            print("TURN: ", i)

            # Agent submit
            _, batch_id = self.submit_batch(agent_name)

            # Retrieve
            file_id = self.client.batches.retrieve(batch_id).output_file_id
            content = self.client.files.content(file_id)

            agent_res, client_res = self.load_format(content)

            self.append_jsonl(agent_name, agent_res)
            self.append_jsonl(client_name, client_res)
            # FOR LOOP HAULT OPERATION HERE MIGUEL 8/5/24
            #ipdb.set_trace()
            message_content = agent_res[0]['response']['body']['choices'][0]['message']['content']
            if "Adiós" in message_content or "Hasta" in message_content:
              print(f"Ending phrase in conversation detected. Breaking to new convo.")
              break
            # Client submit
            _, batch_id = self.submit_batch(client_name) # ERROR HERE

            # Retrieve
            file_id = self.client.batches.retrieve(batch_id).output_file_id
            content = self.client.files.content(file_id)

            client_res, agent_res = self.load_format(content)

            self.append_jsonl(agent_name, agent_res)
            self.append_jsonl(client_name, client_res)

In [ ]:
    %%add_to self_talk
    def load_format(self, content):
        """
        - Extracts OpenAI's http response into jsonl format
        """
        normal = []
        inverted = []

        for x in content.text.split('\n'):
            if x:
                normal.append(json.loads(x))

        inverted = copy.deepcopy(normal)

        for x in inverted:
            x["response"]["body"]["choices"][0]["message"]["role"] = "user"

        return normal, inverted #normal is agent_res, inverted is client_res FOR AGENT. for client it is opposite

In [ ]:
    %%add_to self_talk
    def submit_batch(self, name):
        """
        - Submit jsonl and then await for completion or hang.
        """
        self.change_dir()
        print(f"Attempting to submit batch: {name}\n")

        batch_input_file = self.client.files.create(
        file=open(f"{name}", "rb"),
        purpose="batch"
        )

        batch_input_file_id = batch_input_file.id

        batch_obj = self.client.batches.create(
            input_file_id=batch_input_file_id,
            endpoint="/v1/chat/completions",
            completion_window="24h",
            metadata={
            "description": "LEIDOS job"
            }
        )

        print(batch_input_file_id, "\n"*4, batch_obj)
        time.sleep(10)

        while self.status(batch_obj.id) != 'completed':
                currenttime=time.strftime('%m-%d-%y_%H%M-%S', time.localtime(time.time())) #changed 8/4/24
                print(currenttime, batch_obj.id, self.status(batch_obj.id))
                self.timeout()

        return batch_obj, batch_obj.id

In [ ]:
if __name__=="__main__":
    """Creating batches"""
    process = self_talk(top_p=0.7,temp=0.8,presence=0.1,PATH="/content/drive/MyDrive/Leidos/LEIDOS/",key="sk")
    #PATH="E:/LEIDOS/"



    #UID = process.create_batch(50, "Hola.") #CHANGED FROM 'NO RESPONDO PREGUNTAS.' - this # is how many prompts generated to send to generate OVERRIDES FUNCTION INPUT IF SPECIFIED HERE
    UID = process.create_batch(200)
    process.generate_batches(UID, utterances=10) # utterances are "turns" in the conversations

    # process.cancel_batch('batch_8nHUSrI9pgIfgq9AIJcBJE5W')

    """Batch Submission"""
    # UID = f"agent_request_{UID}" #  ML 8/4/24
    # process.submit_batch(UID)
    """Check Status"""
    # batch_id = "batch_KUzMeJxc7cc9DQKm1zWgrE45"
    # print(process.status(batch_id))


    """Single API Requests"""
    #process.start_generation(f"August5Activity", 1,10) # 2 total conversations 4 turns each

    # threads = []
    # amt_threads = 2

    # for i in range(amt_threads):
    #     threads.append(threading.Thread(target=process.start_generation, args=(f"T_{i}_buenastardes_", 200,6)))

    # for t in threads:
    #     t.start()

    # for t in threads:
    #     t.join()

    # print("Threads Joined.")

    """Print out content"""
    #process.print_content("agent_request_1720039424414.jsonl")

    """DB CREATION"""
    # process.create_db("unfiltered", [1716484007823, 1716519250525, 1716586796027, 1716586883138, 1716656236989, 1716656246191])


/content

|
/content/drive/MyDrive/Leidos/LEIDOS/batched
Requested 200 prompts to be generated!
Generated prompt: 0!
Generated prompt: 1!
Generated prompt: 2!
Generated prompt: 3!
Generated prompt: 4!
Generated prompt: 5!
Generated prompt: 6!
Generated prompt: 7!
Generated prompt: 8!
Generated prompt: 9!
Generated prompt: 10!
Generated prompt: 11!
Generated prompt: 12!
Generated prompt: 13!
Generated prompt: 14!
Generated prompt: 15!
Generated prompt: 16!
Generated prompt: 17!
Generated prompt: 18!
Generated prompt: 19!
Generated prompt: 20!
Generated prompt: 21!
Generated prompt: 22!
Generated prompt: 23!
Generated prompt: 24!
Generated prompt: 25!
Generated prompt: 26!
Generated prompt: 27!
Generated prompt: 28!
Generated prompt: 29!
Generated prompt: 30!
Generated prompt: 31!
Generated prompt: 32!
Generated prompt: 33!
Generated prompt: 34!
Generated prompt: 35!
Generated prompt: 36!
Generated prompt: 37!
Generated prompt: 38!
Generated prompt: 39!
Generated prompt: 40!
Generated pr